# Predictive Model Development

In [64]:
# import neccessary libraries
import numpy as np
import pandas as pd

In [4]:
#load data to dataframe
df=pd.read_csv("accident_2020_data.csv")

C:\Users\PC\AppData\Local\Temp\ipykernel_7864\3553628271.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("accident_2020_data.csv")


In [8]:
# Step 2: Map strings to integers
severity_mapping = {
    'Slight': 1,
    'Serious': 2,
    'Fatal': 3
}
df['accident_severity'] = df['accident_severity'].map(severity_mapping)

# Step 3: Convert the column to integer type
df['accident_severity'] = df['accident_severity'].astype(int)

#### from the EDA we did last, we represent the categorical values with slight, serious and fatal but as regards this predictive modeling, we will undo it

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91185 entries, 0 to 91184
Data columns (total 41 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   accident_index                               91185 non-null  object 
 1   accident_year                                91185 non-null  int64  
 2   accident_reference                           91185 non-null  object 
 3   location_easting_osgr                        91185 non-null  float64
 4   location_northing_osgr                       91185 non-null  float64
 5   longitude                                    91185 non-null  float64
 6   latitude                                     91185 non-null  float64
 7   police_force                                 91185 non-null  int64  
 8   accident_severity                            91185 non-null  int32  
 9   number_of_vehicles                           91185 non-null  int64  
 10

In [11]:
#select the numeric data
numeric_df = df.select_dtypes(include=['int', 'float'])
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91185 entries, 0 to 91184
Data columns (total 34 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   accident_year                                91185 non-null  int64  
 1   location_easting_osgr                        91185 non-null  float64
 2   location_northing_osgr                       91185 non-null  float64
 3   longitude                                    91185 non-null  float64
 4   latitude                                     91185 non-null  float64
 5   police_force                                 91185 non-null  int64  
 6   accident_severity                            91185 non-null  int32  
 7   number_of_vehicles                           91185 non-null  int64  
 8   number_of_casualties                         91185 non-null  int64  
 9   day_of_week                                  91185 non-null  int64  
 10

In [13]:
# Replace 2,3 with 1 and 1 with 0(fatal & severe accident =1, while slight accident=0)
numeric_df['accident_severity'] = numeric_df['accident_severity'].replace({2: 1, 3: 1, 1: 0})

In [17]:
numeric_df["accident_severity"].value_counts()

accident_severity
0    71441
1    19744
Name: count, dtype: int64

In [19]:
#Drop irrelevant columns
numeric_df.drop(columns=["Day","Month","Year","hour","season","accident_year"], inplace=True)

In [40]:
# Loop through the columns to find and drop binary columns, except 'accident_severity'
for column in numeric_df.columns:
    if column != 'accident_severity':  # Exclude 'accident_severity'
        unique_values = numeric_df[column].nunique()
        if unique_values == 2:  # Check if the column is binary
            numeric_df.drop(column, axis=1, inplace=True)

# Display the remaining columns
numeric_df.columns

Index(['location_easting_osgr', 'location_northing_osgr', 'longitude',
       'latitude', 'police_force', 'accident_severity', 'number_of_vehicles',
       'number_of_casualties', 'day_of_week', 'local_authority_district',
       'first_road_class', 'first_road_number', 'road_type', 'speed_limit',
       'junction_detail', 'junction_control', 'second_road_class',
       'second_road_number', 'pedestrian_crossing_human_control',
       'pedestrian_crossing_physical_facilities', 'light_conditions',
       'weather_conditions', 'road_surface_conditions',
       'special_conditions_at_site', 'carriageway_hazards',
       'did_police_officer_attend_scene_of_accident'],
      dtype='object')

In [41]:
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91185 entries, 0 to 91184
Data columns (total 26 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   location_easting_osgr                        91185 non-null  float64
 1   location_northing_osgr                       91185 non-null  float64
 2   longitude                                    91185 non-null  float64
 3   latitude                                     91185 non-null  float64
 4   police_force                                 91185 non-null  int64  
 5   accident_severity                            91185 non-null  int32  
 6   number_of_vehicles                           91185 non-null  int64  
 7   number_of_casualties                         91185 non-null  int64  
 8   day_of_week                                  91185 non-null  int64  
 9   local_authority_district                     91185 non-null  int64  
 10

## Feature Scaling

In [49]:
from sklearn.preprocessing import StandardScaler

In [42]:
from sklearn.preprocessing import MinMaxScaler

In [50]:
# Features Scaling
# Initialize the StandardScaler
scaler = StandardScaler()

# Identify columns to scale (all columns except 'accident_severity')
columns_to_scale = numeric_df.columns.difference(['accident_severity'])

# Apply the scaler to these columns
numeric_df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
numeric_df

,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,day_of_week,local_authority_district,...,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,did_police_officer_attend_scene_of_accident
0,0.694039,-0.669291,0.683778,-0.669505,-1.078921,0,-1.233225,-0.392354,-0.580402,-1.212492,...,-1.110512,-0.240999,5.093192,3.195529,-0.609585,3.954403,8.305448,-0.187633,-0.159251,2.381465
1,0.779033,-0.661873,0.767672,-0.663430,-1.078921,0,-1.233225,1.074992,-1.097940,-1.216470,...,1.074614,-0.240999,-0.208901,1.150543,-0.609585,-0.380345,-0.435464,-0.187633,-0.159251,-0.624063
2,0.747968,-0.617598,0.738818,-0.618771,-1.078921,0,-1.233225,-0.392354,-0.062864,-1.248300,...,1.074614,-0.240999,-0.208901,-0.485446,1.107114,-0.380345,0.657150,-0.187633,-0.159251,-0.624063
3,0.878903,-0.606671,0.868251,-0.610036,-1.078921,1,-1.233225,-0.392354,-0.062864,-1.184641,...,-1.110512,-0.240999,-0.208901,1.150543,1.107114,-0.380345,-0.435464,-0.187633,-0.159251,-0.624063
4,0.778894,-0.627608,0.768886,-0.629249,-1.078921,0,-1.233225,1.074992,-0.062864,-1.248300,...,0.710426,-0.240999,-0.208901,-0.485446,1.107114,-0.380345,-0.435464,-0.187633,-0.159251,-0.624063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91180,-1.213252,3.107480,-1.269988,3.105282,2.913298,1,0.243235,-0.392354,-0.062864,2.400187,...,0.346239,0.808679,-0.208901,-0.485446,-0.609585,-0.380345,-0.435464,-0.187633,-0.159251,-0.624063
91181,-2.122984,2.613672,-2.250588,2.599706,2.913298,0,-1.233225,-0.392354,0.972212,2.416101,...,1.074614,-0.240999,-0.208901,-0.485446,-0.609585,-0.380345,-0.435464,-0.187633,-0.159251,0.878701
91182,-0.778763,3.643561,-0.791535,3.642188,2.913298,1,0.243235,-0.392354,-0.062864,2.368357,...,1.074614,-0.240999,-0.208901,-0.485446,-0.609585,-0.380345,-0.435464,-0.187633,-0.159251,-0.624063
91183,-1.917684,2.721992,-2.032123,2.711630,2.913298,0,0.243235,-0.392354,-0.580402,2.455889,...,1.074614,-0.240999,-0.208901,-0.485446,-0.609585,-0.380345,-0.435464,-0.187633,-0.159251,0.878701


## Handling imbalance Data

In [52]:
X=numeric_df.drop(columns=["accident_severity"],axis=1)
y=numeric_df["accident_severity"]

In [53]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [73]:
# Initialize SMOTE
random_over_samp = RandomOverSampler(random_state=42)

# Fit and apply SMOTE to the training data
X_res, y_res = random_over_samp.fit_resample(X, y)

# Check the new class distribution
print("Original class distribution:\n", y.value_counts())
print("Resampled class distribution:\n", y_res.value_counts())

Original class distribution:
 accident_severity
0    71441
1    19744
Name: count, dtype: int64
Resampled class distribution:
 accident_severity
0    71441
1    71441
Name: count, dtype: int64


## Split Data

In [70]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

## Build Model

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [77]:
# Algorithm with the best model
model_ens = ExtraTreesClassifier(random_state=42)
model_ens.fit(X_train, y_train)

ExtraTreesClassifier(random_state=42)

In [78]:
# Make predictions and evaluate
y_pred = model_ens.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93     21344
           1       0.92      0.94      0.93     21521

    accuracy                           0.93     42865
   macro avg       0.93      0.93      0.93     42865
weighted avg       0.93      0.93      0.93     42865

